In [1]:
import requests
import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', 1000)

In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from scipy.special import softmax
import tensorflow as tf

2024-04-20 05:51:17.488921: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 05:51:17.489072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 05:51:17.662986: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
twitter_data = []

In [4]:
payload = {
    'api_key': '4a4d79dd707289be420741542b93fd0f',
    'query': 'I am depressed',
    'num': '10'
}

response = requests.get('https://api.scraperapi.com/structured/twitter/search', params=payload)
data = response.json()

In [5]:
print(data['organic_results'][0]['snippet'])

Bad responses: “I am depressed” - Shake it off. “I am depressed” - You're fine. “I am depressed” - It's in your head.


In [6]:
all_tweets = data['organic_results']
for tweet in all_tweets:
    twitter_data.append(tweet)

In [7]:
tweet_df = pd.DataFrame(twitter_data)
tweet_df.head(10)

,position,title,snippet,highlighs,link,displayed_link
0,0,"Dr. Cameron Ghassemi, DO on X: ""Bad responses",Bad responses: “I am depressed” - Shake it off. “I am depressed” - You're fine. “I am depressed” - It's in your head.,"[I am depressed, I am depressed, I am depressed]",https://twitter.com/camsghassemi/status/1416967711742177281,https://twitter.com › camsghassemi › status
1,1,"Skyler Higley on X: ""I hate the idea that “I” am depressed. ...","Conversation ... I hate the idea that “I” am depressed. Everything's fucked up, why are we putting this on me like I'm wrong ???",[I” am depressed],https://twitter.com/skyler_higley/status/1678506713937330178,https://twitter.com › skyler_higley › status
2,2,"Lenora Houseworth on X: ""I be thinking I am depressed",I be thinking I am depressed; the whole time I just needed to express my creativity without capitalism attached to it.,[I am depressed],https://twitter.com/LenoraSheWrote/status/1775541640674894207,https://twitter.com › LenoraSheWrote › status
3,3,Agnes C. Poirier,I am depressed!,[I am depressed],https://twitter.com/AgnesCPoirier/status/1662871925347983360,https://twitter.com › AgnesCPoirier › status
4,4,Helene von Bismarck,"And the reason I am depressed is not because of one weapon system Berlin decided not to send. It is because instead of sticking together, ...",[I am depressed],https://twitter.com/HeleneBismarck/status/1763288916906438938,https://twitter.com › HeleneBismarck › status
5,5,I Am Depressed (@DepressedLow) / ...,"I Am Depressed's posts ... My depression is undiagnosed, unseen, and unreal to the people who surround me. I know if I ever get the guts to tell them they'll ...",[I Am Depressed's],https://twitter.com/depressedlow,https://twitter.com › depressedlow
6,6,"UNC Barstool on X: ""I am depressed"" / X","Conversation. UNC Barstool · @UNCBarstool. I am depressed. 4:03 AM · Mar 29, 2024. ·. 54.5K. Views. 264. Reposts.",[I am depressed],https://twitter.com/UNCBarstool/status/1773561632850887091,https://twitter.com › UNCBarstool › status
7,7,I am depressed now... because I know that I will never ...,I am depressed now... because I know that I will never make anything as funny as this video: http://t.co/u2qBHMQk.,[I am depressed],https://twitter.com/markiplier/status/300720891438956544,https://twitter.com › markiplier › status
8,8,Big Jim Miller,"Conversation. Big Jim Miller · @TheBiggestJim. I am depressed. 1:04 AM · Mar 6, 2024. ·. 1,382. Views.",[I am depressed],https://twitter.com/TheBiggestJim/status/1765181628836659274,https://twitter.com › TheBiggestJim › status
9,9,Phil Elverum - I am depressed.,I am depressed.,[I am depressed],https://twitter.com/PWElverum/status/946956073818210306,https://twitter.com › PWElverum › status


In [18]:
tweet_test_sad = tweet_df['snippet'][0]
tweet_test_happy = "The weather is beautiful"

In [19]:
print(tweet_test_sad)

Bad responses: “I am depressed” - Shake it off. “I am depressed” - You're fine. “I am depressed” - It's in your head.


In [20]:
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = TFAutoModelForSequenceClassification.from_pretrained(roberta)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [21]:
tokenizer = AutoTokenizer.from_pretrained(roberta)

In [22]:
labels = ['Negative', 'Neutral', 'Positive']

In [27]:
encoded_tweet = tokenizer(tweet_test_sad, return_tensors='tf')
output = model(encoded_tweet)

In [28]:
scores = output[0][0].numpy()
scores = softmax(scores)
score_dict = {}

In [29]:
for i in range(len(scores)):
    l = labels[i]
    s = scores[i]
    print(l,s)
    score_dict[l] = s

Negative 0.6051382
Neutral 0.33306706
Positive 0.061794724


In [30]:
# This is not accurate enough we should add the scores and then conclude if the user is really sad
# this cant detect sarcasm and contradictory sentences
max_key = max(score_dict, key=score_dict.get)
print(f"This tweet is {max_key}")

This tweet is Negative
